This notebook adds the VC keyword matches to the index file.

# Imports

In [57]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.util import ngrams
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

# Setup

## Load our list of keywords created in keywords.py

In [58]:
with open("keywords.pkl", "rb") as f:
    keywords = pickle.load(f)        

In [59]:
print(keywords[14])

['navig navig safeti', 'navig', 'nonnavig', 'navig protect activ', 'guid line', 'guid wire', 'signag', 'waterway', 'cross', 'cross plan', 'impact navig', 'dewat navig water', 'navig water', 'navig waterway', 'navig', 'navig', 'watercours', 'watercours cross', 'water cross', 'cross methodolog', 'horizont direct drill', 'hdd', 'bridg', 'marin', 'marin termin', 'waterway user', 'recreat waterway user', 'navig use', 'tributari', 'tourism', 'tourist', 'guid outfitt', 'outfitt', 'angler', 'cano', 'kayak', 'boat', 'sail', 'sail', 'fisheri ocean canada', 'navig protect act', 'navig watercours', 'navig safeti', 'schedul water', 'non-schedul water', 'recreation-rel navig', 'commercial-rel navig', 'watercours user', 'waterway user', 'watercours cross', 'cross method', 'trenchless cross', 'trench cross', 'navig hazard', 'expos instream', 'buoyanc issu', 'upstream', 'downstream', 'warn sign', 'warn signag', 'instream', 'temporari vehicl cross', 'bed', 'bank', 'preconstruct contour', 'hydraul charac

In [60]:
# list of VCs

labels_list = ['Landscape, terrain, and weather', 
                'Soil',
                'Plants',
                'Water',
                'Fish',
                'Wetlands',
                'Wildlife',
                'Species at Risk',
                'Greenhouse gas emissions',
                'Air emissions',
                'Noise',
                'Electricity and electromagnetism',
                'Proximity to people',
                'Archaeological, paleontological, historical, and culturally significant sites and resources',
                'Human access to boats and waterways',
                'Indigenous land, water, and air use',
                'Impact to social and cultural well-being',
                'Impact to human health and viewscapes',
                'Social, cultural, economic infrastructure and services',
                'Economic Offsets and Impact',
                'Environmental Obligations',
                'Treaty and Indigenous Rights']

# Load the main index file for old projects

In [115]:
# df = pd.read_csv('./data/ESA_website_ENG_with_VCs_onehot.csv')
df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=0)

C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12,23,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [116]:
df.drop(columns=['label'], inplace=True)

In [117]:
df.head(2)

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614.0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14.0,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614.0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17.0,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...


# Adding New Projects to the Old Index File

In [130]:
new_projects_df = pd.read_csv('./data/new_projects_index_file.csv')
new_projects_df.head(2)

,PDF Page Number,tableNumber,Title,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing Order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,PDF Page Count,Content Type,ESA Section(s) Topics,Hearing order,PDF Outline,PDF Size,Download folder name,filename,CSV Download URL,old_filename,text
0,12,1,Table 1: Watercourses Crossed by the Raven River Section,Application for the NGTL West Path Delivery 2022 Project,West Path Delivery 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gas,C06605-8 Attachment 17 - Environmental Assessment Summary - A7G0Q1,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3970826,A7G0Q1,3931508,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3931508,CERA 183 Pipeline Facilities Certificate,Alberta,NaN,Stantec Consulting Ltd,Approved,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,Environmental Assessment Summary,NaN,16,Table,NaN,NaN,Yes,0.2,wstpthdlvr2022,wstpthdlvr2022_table-1-watercourses-crossed-by-the-raven-river-se_pt-1_pg-12_doc-num-A7G0Q1.csv,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022/wstpthdlvr2022_table-1-watercourses-crossed-by-t...,3931508_12_lattice-v_1.csv,"﻿""Crossing ID"" Watercourse Name Legal Location Mean Channel Width (m) Fish Habitat Fish Habitat ..."
1,13,1,Table 1: Watercourses Crossed by the Raven River Section,Application for the NGTL West Path Delivery 2022 Project,West Path Delivery 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gas,C06605-8 Attachment 17 - Environmental Assessment Summary - A7G0Q1,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3970826,A7G0Q1,3931508,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3931508,CERA 183 Pipeline Facilities Certificate,Alberta,NaN,Stantec Consulting Ltd,Approved,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,Environmental Assessment Summary,NaN,16,Table,NaN,NaN,Yes,0.2,wstpthdlvr2022,wstpthdlvr2022_table-1-watercourses-crossed-by-the-raven-river-se_pt-2_pg-13_doc-num-A7G0Q1.csv,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022/wstpthdlvr2022_table-1-watercourses-crossed-by-t...,3931508_13_lattice-v_1.csv,"﻿""Crossing ID"" Watercourse Name Legal Location Mean Channel Width (m) Fish Habitat Fish Habitat ..."


In [131]:
def add_index_new_projects(old_df, new_projects_df):
    """
    Add an index column to a dataframe starting at len(old_df).
    """
    new_projects_index = list(range(old_df['Index'].iloc[-1] + 1, old_df['Index'].iloc[-1] + 1 + len(new_projects_df)))
    new_projects_df['Index'] = new_projects_index

add_index_new_projects(df, new_projects_df)

In [132]:
from numpy import NaN

new_projects_df['Content Type'] = 'Table'
new_projects_df['ESA Section(s) Topics'] = NaN
new_projects_df['Zipped Project Link'] = NaN
new_projects_df['Missing CSV'] = bool(False)
new_projects_df.rename(columns={'filename': 'CSV Filename'}, inplace=True)
new_projects_df.rename(columns={'old_filename': 'csvFileName'}, inplace=True)
# new_projects_df.rename(columns={'Hearing Order': 'Hearing order'}, inplace=True)
new_projects_df.drop(columns=['tableNumber'], inplace=True)
new_projects_df['text'] = new_projects_df['Title'] + ' ' + new_projects_df['text']

In [133]:
for i, row in enumerate(new_projects_df.itertuples()):
    new_projects_df['Zipped Project Link'][i] = 'http://www.cer-rec.gc.ca/esa-ees/' + str(new_projects_df['Download folder name'][i]) + '.zip'

<ipython-input-133-a6412208edb5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_projects_df['Zipped Project Link'][i] = 'http://www.cer-rec.gc.ca/esa-ees/' + str(new_projects_df['Download folder name'][i]) + '.zip'
C:\Users\ThibJacq\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [134]:
new_projects_df.index += df['Index'].iloc[-1] + 1
new_projects_df.head(2)

,PDF Page Number,Title,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing Order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,PDF Page Count,Content Type,ESA Section(s) Topics,Hearing order,PDF Outline,PDF Size,Download folder name,CSV Filename,CSV Download URL,csvFileName,text,Index,Zipped Project Link,Missing CSV
38025,12,Table 1: Watercourses Crossed by the Raven River Section,Application for the NGTL West Path Delivery 2022 Project,West Path Delivery 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gas,C06605-8 Attachment 17 - Environmental Assessment Summary - A7G0Q1,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3970826,A7G0Q1,3931508,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3931508,CERA 183 Pipeline Facilities Certificate,Alberta,NaN,Stantec Consulting Ltd,Approved,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,Environmental Assessment Summary,NaN,16,Table,NaN,NaN,Yes,0.2,wstpthdlvr2022,wstpthdlvr2022_table-1-watercourses-crossed-by-the-raven-river-se_pt-1_pg-12_doc-num-A7G0Q1.csv,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022/wstpthdlvr2022_table-1-watercourses-crossed-by-t...,3931508_12_lattice-v_1.csv,"Table 1: Watercourses Crossed by the Raven River Section ﻿""Crossing ID"" Watercourse Name Legal L...",38025,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022.zip,False
38026,13,Table 1: Watercourses Crossed by the Raven River Section,Application for the NGTL West Path Delivery 2022 Project,West Path Delivery 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gas,C06605-8 Attachment 17 - Environmental Assessment Summary - A7G0Q1,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3970826,A7G0Q1,3931508,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/3931508,CERA 183 Pipeline Facilities Certificate,Alberta,NaN,Stantec Consulting Ltd,Approved,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3930580,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4091795,Environmental Assessment Summary,NaN,16,Table,NaN,NaN,Yes,0.2,wstpthdlvr2022,wstpthdlvr2022_table-1-watercourses-crossed-by-the-raven-river-se_pt-2_pg-13_doc-num-A7G0Q1.csv,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022/wstpthdlvr2022_table-1-watercourses-crossed-by-t...,3931508_13_lattice-v_1.csv,"Table 1: Watercourses Crossed by the Raven River Section ﻿""Crossing ID"" Watercourse Name Legal L...",38026,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2022.zip,False


In [139]:
df = pd.concat([df, new_projects_df], axis=0, ignore_index=True)
df.drop(columns=['Hearing Order'], inplace=True)

In [197]:
table_texts = df['text'].tolist()

tokenized_table_texts = []
stemmer = PorterStemmer()

for i, table_text in enumerate(table_texts):
    processed_text = word_tokenize(table_text)
    processed_text = [stemmer.stem(w) for w in processed_text if w not in stopwords.words("english")]
    table_ngram_list = []
    for n in range(1, 7):
        table_ngrams = list(ngrams(processed_text, n))
        table_ngram_list.extend([" ".join(table_gram) for table_gram in table_ngrams])

    tokenized_table_texts.append(table_ngram_list)
    
print(tokenized_table_texts[0][0:200])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field', 'studi', 'particip', 'for', 'the', 'project', 'survey', 'date', 'aborigin', 'commun', 'detail', 'juli', '9', '12', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'summer', '2011', 'open', 'water', 'aquat', 'assess', 'septemb', '18', 'octob', '10', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'fall', '2011', 'open', 'water', 'aquat', 'assess', 'februari', '26', 'march', '12', '2012', 'blueberri', 'river', 'first', 'nation', 'doig', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'kelli', 'lake', 'cree', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'winter', '2012', 'aquat', 'habitat', 'investig', 'tabl 3', '3 summari', 's

In [199]:
with open('tokenized_table_texts.pkl', 'wb') as f:
    pickle.dump(tokenized_table_texts, f)

In [45]:
with open('tokenized_table_texts.pkl', 'rb') as f:
    tokenized_table_text = pickle.load(f)

In [52]:
print(tokenized_table_text[0][:10], '\n', tokenized_table_text[2932][:50])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field'] 
 ['tabl', '510', 'navig', 'classif', 'OF', 'the', 'potenti', 'watercours', 'cross', 'investig', 'IN', 'the', 'black', 'pine', 'TO', 'hope', 'segment', 'classif', 'total', 'number', 'class', '1', 'nonnavig', '23', 'class', '2', 'nonnavig', '43', 'class', '3', 'nonnavig', '15', 'potenti', 'navig', '14', 'navig', '15', 'tabl 510', '510 navig', 'navig classif', 'classif OF', 'OF the', 'the potenti', 'potenti watercours', 'watercours cross', 'cross investig', 'investig IN', 'IN the', 'the black', 'black pine']


In [51]:
for label in labels_list:
    df[f'{label}'] = 0
    # df[f'{label} - Number of Matches'] = 0
    # df[f'{label} Relevance'] = 0

df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
def label_with_keywords(i, table_text, keywords_for_label, label):
    number_of_matches = len([word for word in keywords_for_label if word in table_text])
    df[f'{label}'][i] = number_of_matches

In [54]:
def string_total_sum_match_2_lists(list_1, list_2):
    """
    This function takes two lists of strings and counts the number of total matches between the two lists, including duplicates.
    The function returns the number of total matches.
    """
    # Initialize the count variable.
    count = 0
    # Iterate over the first list.
    for item in list_1:
        # If the item is in the second list, increment the count variable.
        if item in list_2:
            count += 1
    # Return the count variable.
    return count

In [55]:
df.text[0]

'TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Summer 2011 open water aquatic assessments September 18 to October 10 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Fall 2011 open water aquatic assessments February 26 to March 12 2012 Blueberry River First Nation Doig River First Nation Halfway River First Nation Kelly Lake Cree First Nation McLeod Lake Indian Band North East Mtis Association Winter 2012 aquatic habitat investigations '

In [56]:
for i, table_text in enumerate(tokenized_table_text):
    for keywords_for_label, label in zip(keywords, labels_list):
        number_of_matches = string_total_sum_match_2_lists(table_text, keywords_for_label)
        df[f'{label}'][i] = number_of_matches
        
with open("esa_index_ENG_labeled.pkl", "wb") as f:
    pickle.dump(df, f)

df.head()

<ipython-input-56-94bef19dd2c9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = number_of_matches


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,3,2,6,47,89,3,13,16,1,0,0,0,0,0,25,22,2,0,1,20,2,22
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-w

In [57]:
df.to_csv('./mydata/esa_index_keyword_labeled_3aug2021.csv')

In [58]:
df = pd.read_csv('./mydata/esa_index_keyword_labeled_3aug2021.csv')

In [59]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [60]:
df.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,3,2,6,47,89,3,13,16,1,0,0,0,0,0,25,22,2,0,1,20,2,22
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-w

In [18]:
# with open("esa_index_ENG_labeled.pkl", "rb") as f:
#     df = pickle.load(f)

In [106]:
from sklearn.model_selection import train_test_split
train_df, validation_df = train_test_split(df, test_size=300, random_state=42)
validation_df_labels = validation_df.iloc[:, 33:]
validation_df_labels.head(2)

,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
8889,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,10,1,2,0,4,2,1,2,1,0,12,4,0,4
27753,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,0,0,0,2,0,1,0,2,0,1,0,10,4,1,1


In [62]:
validation_df_labels['Comments'] = 'N/A'

In [63]:
train_df.to_csv("./data/esa_keyword_labeled_train_df_3aug2021.csv", index=False)
validation_df.to_csv("./data/esa_keyword_labeled_validation_3aug2021df.csv", index=False)
validation_df_labels.to_csv("./data/esa_keyword_labeled_validation_df_labels_3aug2021.csv", index=False)

In [107]:
ground_truth_val_df = pd.read_excel('./data/esa_keyword_labeled_validation_df_labels.xlsx')

In [108]:
ground_truth_val_df.head()

,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,0,2,2,0,4,2,1,2,1,0,11,9,0,4
1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,13,0,0,0,0,1,0,6,0,1,0,6,4,0,1
2,TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIES BY TYPE AND COMMON NAME Common Name Scientific N...,0,0,78,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TABLE 32 CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES Stakeholder Group Agency Name Name and...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Table 5218 Major Proposed Approved and Recently Completed Projects in the SocioEconomic Study Ar...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,0,0,0


In [109]:
validation_df.reset_index(inplace=True)

In [110]:
validation_df.iloc[:, 34:].head()

,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,10,1,2,0,4,2,1,2,1,0,12,4,0,4
1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,0,0,0,2,0,1,0,2,0,1,0,10,4,1,1
2,TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIES BY TYPE AND COMMON NAME Common Name Scientific N...,0,0,78,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TABLE 32 CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES Stakeholder Group Agency Name Name and...,1,1,1,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,12,4,1,2
4,Table 5218 Major Proposed Approved and Recently Completed Projects in the SocioEconomic Study Ar...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,0,0,0


In [111]:
validation_df.iloc[:, 34:] = ground_truth_val_df

C:\Users\ThibJacq\Anaconda3\lib\site-packages\pandas\core\indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [115]:
ground_truth_val_df.head(2)

,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,0,2,2,0,4,2,1,2,1,0,11,9,0,4
1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,13,0,0,0,0,1,0,6,0,1,0,6,4,0,1


In [114]:
validation_df.head(2)

,index,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/2748481,A4K2S0,2758825,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/2758825,Large Projects (over 40 km),"Alberta, All",GH-002-2015,TERA a CHTM HILL Company,Operating,GC-126,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2758964,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2985011,Section 8: Cumulative Effects Assessment,15.0,"Other, All",http://www.cer-rec.gc.ca/esa-ees/2017ngtl/2017ngtl_table-8a-2-reasonably-forseeable-developments...,359,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-developments-iden_pt-10_pg-359_doc-num-A4K2S0.csv,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,0,2,2,0,4,2,1,2,1,0,11,9,0,4
1,27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/684358,A1Y9J1,684515,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/684515,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-2-2011,TERA Environmental Consultants,Operating,GC-119,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/685859,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/793577,Appendix 8: Environmental Protection Plan,23.0,"Environment Protection Plan, All",http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln/nrthwstmnln_table-1-resource-specific-mitigation-ta...,22,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigation-table_pt-3_pg-22_doc-num-A1Y9J1.csv,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,13,0,0,0,0,1,0,6,0,1,0,6,4,0,1


In [119]:
validation_df.set_index(list(validation_df)[0], inplace=True)

AttributeError: can't delete attribute

In [120]:
validation_df.index.name = None

In [121]:
validation_df.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/2748481,A4K2S0,2758825,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/2758825,Large Projects (over 40 km),"Alberta, All",GH-002-2015,TERA a CHTM HILL Company,Operating,GC-126,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2758964,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2985011,Section 8: Cumulative Effects Assessment,15.0,"Other, All",http://www.cer-rec.gc.ca/esa-ees/2017ngtl/2017ngtl_table-8a-2-reasonably-forseeable-developments...,359,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-developments-iden_pt-10_pg-359_doc-num-A4K2S0.csv,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEA...,2,1,1,3,3,1,2,0,0,2,2,0,4,2,1,2,1,0,11,9,0,4
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/684358,A1Y9J1,684515,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/684515,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-2-2011,TERA Environmental Consultants,Operating,GC-119,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/685859,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/793577,Appendix 8: Environmental Protection Plan,23.0,"Environment Protection Plan, All",http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln/nrthwstmnln_table-1-resource-specific-mitigation-ta...,22,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigation-table_pt-3_pg-22_doc-num-A1Y9J1.csv,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Location Issues Timing Windows and or Regulatory Guide...,0,3,39,0,0,0,0,13,0,0,0,0,1,0,6,0,1,0,6,4,0,1
4165,13299,Table,Application for Trans Mountain Expansion Project,Trans Mountain Expansion,12/16/2013,Trans Mountain Pipeline ULC,Oil,B9-23 - V5C_TR_5C9_17of20_VEGETATION - A3S2L6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/2385502,A3S2L6,2393491,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/2393491,Large Projects (over 40 km),"Alberta, British Columbia, All",OH-001-2014,"BGC Engineering Inc., Waterline Resources Inc., TERA Environmental Consultants, Mentiga Pedology...",Approved,OC-064. OC-065,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2392873,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3751789,Vegetation Technical Report,161.0,"Vegetation, Technology, All",http://www.cer-rec.gc.ca/esa-ees/tmx/tmx_table-d4-hope-to-burnaby-observed-plant-species--b_pt-5...,176,180.0,2.

In [128]:
final_df = pd.concat([train_df, validation_df], axis=0)

In [130]:
final_df.sort_index(inplace=True)

In [133]:
final_df.to_csv('./data/esa_index_vcs_ENG_with_table_contents_3aug2021.csv')

In [136]:
final_df.drop(columns=['text'], inplace=True)

In [137]:
final_df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,3,2,6,47,89,3,13,16,1,0,0,0,0,0,25,22,2,0,1,20,2,22


In [138]:
final_df.to_csv('./data/esa_index_vcs_ENG_3aug2021.csv')

In [156]:
final_df.iloc[:, 33:].sum().sort_values(ascending=False)

Infrastructure and services                                         158639
Water quality and quantity                                          146050
Employment and economy                                              102744
Fish and fish habitat                                                97665
Rights of Indigenous Peoples                                         93681
Vegetation                                                           92857
Soil and soil productivity                                           83658
Wildlife and wildlife habitat                                        82439
Navigation and navigation safety                                     79562
Traditional land and resource use                                    61676
Human occupancy and resource use                                     60808
Social and cultural well-being                                       48013
Species at Risk or Species of Special Status and related habitat     45080
Wetlands                 

In [150]:
import numpy as np
final_df.iloc[:, 33:].select_dtypes(np.number).gt(0).sum(axis=0).sort_values(ascending=False)

Infrastructure and services                                         17571
Water quality and quantity                                          16873
Employment and economy                                              14674
Rights of Indigenous Peoples                                        12645
Fish and fish habitat                                               12512
Vegetation                                                          12419
Wildlife and wildlife habitat                                       12025
Human occupancy and resource use                                    11957
Soil and soil productivity                                          10961
Social and cultural well-being                                       9856
Traditional land and resource use                                    9294
Navigation and navigation safety                                     8782
Species at Risk or Species of Special Status and related habitat     8071
Environmental Obligations             

In [22]:
labeled_df = pd.read_csv('./data/project-1-at-2021-07-09-00-52-898dbd22.csv')

In [23]:
labeled_df

,Unnamed: 0,text,label,id,sentiment,annotator,annotation_id
0,8889,TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BE...,-1,1,"{""choices"": [""Physical and meteorological environment"", ""Soil and soil productivity"", ""Vegetatio...",jacques.thibodeau@cer-rec.gc.ca,1


In [91]:
for i in range(len(labeled_df)):
    chosen_labels = ast.literal_eval(labeled_df.sentiment[i])["choices"]
    for label in labels_list:
        if label in chosen_labels:
            df[f'{label}'][i] = 1

<ipython-input-91-cb7868f289e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = 1


In [92]:
df.iloc[:,36:58]

,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations
0,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0
1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1
2,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
3,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
4,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28886,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,1
28887,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1
28888,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1
28889,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1


In [ ]:
for i in range(len(labeled_df)):
    

In [117]:
str('"choices" ' + str(df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()) + '}').replace('\'', '')

'"choices" [Physical and meteorological environment, Soil and soil productivity, Vegetation, Water quality and quantity, Fish and fish habitat, Wetlands, Wildlife and wildlife habitat, Acoustic environment, Navigation and navigation safety, Traditional land and resource use, Social and cultural well-being, Rights of Indigenous Peoples]}'

In [119]:
type({"choices": ["Physical and meteorological environment", "Soil and oil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment"]})

dict

In [138]:
myDict = {}
myDict["choices"] = df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()

In [139]:
myDict

{'choices': ['Physical and meteorological environment',
  'Soil and soil productivity',
  'Vegetation',
  'Water quality and quantity',
  'Fish and fish habitat',
  'Wetlands',
  'Wildlife and wildlife habitat',
  'Acoustic environment',
  'Navigation and navigation safety',
  'Traditional land and resource use',
  'Social and cultural well-being',
  'Rights of Indigenous Peoples']}

In [141]:
import json

json.dumps(myDict)

'{"choices": ["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Rights of Indigenous Peoples"]}'

In [247]:
import json

with open('./mydata/project-1-at-2021-07-17-04-15-91c3a72e.json', 'r') as f:
    diction = json.load(f)


In [266]:
pd.DataFrame(diction[13:16])

,id,annotations,predictions,file_upload,data,meta,created_at,updated_at,project
0,5,"[{'id': 5, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 9544, 'text': 'Table 5.21-8: Major Proposed, Approved, and Recently Completed Pro...",{},2021-06-08T21:27:06.468686Z,2021-06-08T21:27:06.468698Z,1
1,4,"[{'id': 4, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 22455, 'text': 'TABLE 3.2 CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES Stake...",{},2021-06-08T21:27:06.468603Z,2021-06-08T21:27:06.468611Z,1
2,3,"[{'id': 3, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 4165, 'text': 'TABLE D4 HOPE TO BURNABY OBSERVED PLANT SPECIES ï¿½ BY TYPE AND CO...",{},2021-06-08T21:27:06.468566Z,2021-06-08T21:27:06.468574Z,1


In [259]:
import pandas as pd

testing_df = pd.DataFrame(diction)
# testing_df.annotations[0]
with open('./mydata/project-1-at-2021-07-17-04-15-91c3a72e.json', 'r') as f:
    testing_df = pd.read_json(f)

testing_df

,id,annotations,predictions,file_upload,data,meta,created_at,updated_at,project
0,1,"[{'id': 1, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 8889, 'text': 'TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN TH...",{},2021-06-08 21:27:06.468460+00:00,2021-07-10 21:55:17.524780+00:00,1
1,17,"[{'id': 17, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15949, 'text': 'Table B-1 Progress Farrell Creek West Gas Plant B-088-I/094-B-01 ...",{},2021-06-08 21:27:06.469107+00:00,2021-06-08 21:27:06.469114+00:00,1
2,16,"[{'id': 16, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15105, 'text': 'Table 8-19 Exposure Point Concentrations in Marine Organisms Expo...",{},2021-06-08 21:27:06.469073+00:00,2021-06-08 21:27:06.469080+00:00,1
3,15,"[{'id': 15, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 10843, 'text': 'Table 6-3 Recommended Mitigation Measures for Fish and Fish Habit...",{},2021-06-08 21:27:06.469041+00:00,2021-06-08 21:27:06.469048+00:00,1
4,14,"[{'id': 14, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 10810, 'text': 'Table 2-6 Residual Effects on Land Resource Use Project Phase Mi...",{},2021-06-08 21:27:06.469005+00:00,2021-06-08 21:27:06.469012+00:00,1
5,13,"[{'id': 13, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 19786, 'text': 'Table 4-11: Effects and Mitigation Measures Raised at the Round 2...",{},2021-06-08 21:27:06.468972+00:00,2021-06-08 21:27:06.468979+00:00,1
6,12,"[{'id': 12, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 24525, 'text': 'Table 11-13 Site Specific Wildlife Mitigation Measures Recommende...",{},2021-06-08 21:27:06.468936+00:00,2021-06-08 21:27:06.468943+00:00,1
7,11,"[{'id': 11, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 351, 'text': 'Table 21-23 Cumulative Effects Initial Screening ï¿½ Social and Cul...",{},2021-06-08 21:27:06.468904+00:00,2021-06-08 21:27:06.468911+00:00,1
8,10,"[{'id': 10, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': ...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 15394, 'text': 'Table 2 TLRU Information and Mitigation ï¿½ Les Atikamekw de Mana...",{},2021-06-08 21:27:06.468870+00:00,2021-06-08 21:27:06.468877+00:00,1
9,9,"[{'id': 9, 'completed_by': {'id': 1, 'email': 'jacques.thibodeau@cer-rec.gc.ca', 'first_name': '...",[],esa_index_for_labeling.csv,"{'Unnamed: 0': 25965, 'text': 'Table 6-7 Agricultural Soils Ratings ï¿½ Infrastructure Physiogr...",{},2021-06-08 21:27:06.468830+00:00,2021-06-08 21:27:06.468837+00:00,1
